# Setup

## Imports

In [1]:
import pandas as pd
from pyarrow import csv
import plotly.express as px
import plotly.io as pio

## Config

In [3]:
pio.renderers.default = 'notebook_connected'

In [4]:
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 25

# Introduction

To be completed

# Data Ingestion

**Data source**: https://www.kaggle.com/neuromusic/avocado-prices

**Data Dictionary**
- **Date**: The date of the observation
- **AveragePrice**: the average price of a single avocado
- **type**: conventional or organic
- **year**: the year
- **Region**: the city or region of the observation
- **Total Volume**: Total number of avocados sold
- **4046**: Total number of avocados with PLU 4046 sold
- **4225**: Total number of avocados with PLU 4225 sold
- **4770**: Total number of avocados with PLU 4770 sold

**Data Overview**

The table below represents weekly 2018 retail scan data for National retail volume (units) and price. Retail scan data comes directly from retailers’ cash registers based on actual retail sales of Hass avocados. Starting in 2013, the table below reflects an expanded, multi-outlet retail data set. Multi-outlet reporting includes an aggregation of the following channels: grocery, mass, club, drug, dollar and military. The Average Price (of avocados) in the table reflects a per unit (per avocado) cost, even when multiple units (avocados) are sold in bags. The Product Lookup codes (PLU’s) in the table are only for Hass avocados. Other varieties of avocados (e.g. greenskins) are not included in this table.

In [5]:
avocado_path = r"C:\Users\matth\OneDrive\Data\Kaggle\avocado.csv"

In [6]:
arrow_avo = csv.read_csv(avocado_path)

Arrow infers data types for each column and picks up Date as timestamp, which saves the effort of having to do this in pandas as its own operation.  I see that the first columns has no label that needs to be investigated.  After checking the source documentation I see that this represent the index and since we already have that baked into our data structure the column can be removed.

In [7]:
arrow_avo

pyarrow.Table
: int64
Date: timestamp[s]
AveragePrice: double
Total Volume: double
4046: double
4225: double
4770: double
Total Bags: double
Small Bags: double
Large Bags: double
XLarge Bags: double
type: string
year: int64
region: string

After removing the column I can safely convert to pandas for further analysis.

In [8]:
df_avo = arrow_avo.remove_column(0).to_pandas()

In [9]:
df_avo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18249 entries, 0 to 18248
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          18249 non-null  datetime64[ns]
 1   AveragePrice  18249 non-null  float64       
 2   Total Volume  18249 non-null  float64       
 3   4046          18249 non-null  float64       
 4   4225          18249 non-null  float64       
 5   4770          18249 non-null  float64       
 6   Total Bags    18249 non-null  float64       
 7   Small Bags    18249 non-null  float64       
 8   Large Bags    18249 non-null  float64       
 9   XLarge Bags   18249 non-null  float64       
 10  type          18249 non-null  object        
 11  year          18249 non-null  int64         
 12  region        18249 non-null  object        
dtypes: datetime64[ns](1), float64(9), int64(1), object(2)
memory usage: 1.8+ MB


First things first I want to look into the numerical columns to make them into a more human friendly name.  The documentation gives some information on this, but it is not explicit so I will search the PLUs on Google for confirmation.  The result is:

- **PLU 4046**: California Small Hass
- **PLU 4225**: Mexico Large Hass
- **PLU 4770**: California Extra Large Hass

It's actually good we looked into this because as a result we see that PLU 4225 is from Mexico while the other two are from California.  The fact that they are from different geographic regions could certainly have an impact on price and volumnes.

Asides from this there are no null values which will make my life easier.

# Data Cleaning

In [10]:
avo_col_mapper = {'4046': 'Cali Small', '4225': 'Mexico Large', '4770': 'Cali xLarge'}
df_avo.rename(columns=avo_col_mapper, inplace=True)
df_avo.head(10)

,Date,AveragePrice,Total Volume,Cali Small,Mexico Large,Cali xLarge,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,"64,236.62","1,036.74","54,454.85",48.16,"8,696.87","8,603.62",93.25,0.00,conventional,2015,Albany
1,2015-12-20,1.35,"54,876.98",674.28,"44,638.81",58.33,"9,505.56","9,408.07",97.49,0.00,conventional,2015,Albany
2,2015-12-13,0.93,"118,220.22",794.70,"109,149.67",130.50,"8,145.35","8,042.21",103.14,0.00,conventional,2015,Albany
3,2015-12-06,1.08,"78,992.15","1,132.00","71,976.41",72.58,"5,811.16","5,677.40",133.76,0.00,conventional,2015,Albany
4,2015-11-29,1.28,"51,039.60",941.48,"43,838.39",75.78,"6,183.95","5,986.26",197.69,0.00,conventional,2015,Albany
5,2015-11-22,1.26,"55,979.78","1,184.27","48,067.99",43.61,"6,683.91","6,556.47",127.44,0.00,conventional,2015,Albany
6,2015-11-15,0.99,"83,453.76","1,368.92","73,672.72",93.26,"8,318.86","8,196.81",122.05,0.00,conventional,2015,Albany
7,2015-11-08,0.98,"109,428.33",703.75,"101,815.36",80.00,"6,829.22","6,266.85",562.37,0.00,conventional,2015,Albany
8,2015-11-01,1.02,"99,811.42","1,022.15","87,315.57",85.34,"11,388.36","11,104.53",283.83,0.00,conventional,2015,Albany
9,2015-10-25,1.07,"74,338.76",842.40,"64,757.44",113.00,"8,625.92","8,061.47",564.45,0.00,conventional,2015,Albany


Now I will sanity check the values in the columns to see if there are any outliers that need to be handled.

- For the column 'Date' I will look at the range and distribution of dates available
- For integer and float data I will generate summary stats (mean, quartiles, min, max) to get a high level view of the distribution
- For strings I will look at the cardinality and unique values.

For the 'Date' field I can see that the observation period starts on January 4th 2015 and ends on March 25th 2018.  I can also see the distribution across time periods is completely uniform.  This is positive, as it shows consistency in collecting data.  Now we'll need to see if the actual data collected is of good quality.

In [11]:
df_avo.Date.min()

Timestamp('2015-01-04 00:00:00')

In [12]:
df_avo.Date.max()

Timestamp('2018-03-25 00:00:00')

In [15]:
fig = px.histogram(df_avo, x='Date', title='Distribution of Record Dates')
fig.show()

In [18]:
df_avo.describe()

,AveragePrice,Total Volume,Cali Small,Mexico Large,Cali xLarge,Total Bags,Small Bags,Large Bags,XLarge Bags,year
count,"18,249.00","18,249.00","18,249.00","18,249.00","18,249.00","18,249.00","18,249.00","18,249.00","18,249.00","18,249.00"
mean,1.41,"850,644.01","293,008.42","295,154.57","22,839.74","239,639.20","182,194.69","54,338.09","3,106.43","2,016.15"
std,0.40,"3,453,545.36","1,264,989.08","1,204,120.40","107,464.07","986,242.40","746,178.51","243,965.96","17,692.89",0.94
min,0.44,84.56,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,015.00"
25%,1.10,"10,838.58",854.07,"3,008.78",0.00,"5,088.64","2,849.42",127.47,0.00,"2,015.00"
50%,1.37,"107,376.76","8,645.30","29,061.02",184.99,"39,743.83","26,362.82","2,647.71",0.00,"2,016.00"
75%,1.66,"432,962.29","111,020.20","150,206.86","6,243.42","110,783.37","83,337.67","22,029.25",132.50,"2,017.00"
max,3.25,"62,505,646.52","22,743,616.17","20,470,572.61","2,546,439.11","19,373,134.37","13,384,586.80","5,719,096.61","551,693.65","2,018.00"
